In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as mplt
import matplotlib as plt
import datetime as dati

# ¿Podemos predecir los resultados en base a la necesidad/aprobación burocrática del precio?

In [137]:
dataset = pd.read_csv("/home/federico/Git/datos-tp2-2c2020/data/friofrio/Train_TP2_Datos_2020-2C.csv",
        parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                       'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
dataset = dataset.drop(columns= ['Prod_Category_A',
                                 'Product_Category_B',
                                 'Actual_Delivery_Date',
                                 'Submitted_for_Approval',
                                 'Last_Activity','Price',
                                 'Currency','Brand',
                                 'Size','Product_Type'])

In [138]:
dataset.T

,0,1,2,3,4,5,6,7,8,9,...,16937,16938,16939,16940,16941,16942,16943,16944,16945,16946
ID,27761,27760,27446,16808,16805,16802,16799,27455,24353,24355,...,17683,28767,18324,20827,20830,8781,8786,8792,28561,28318
Region,EMEA,EMEA,Americas,Americas,Americas,Americas,Americas,Americas,Japan,Japan,...,EMEA,EMEA,EMEA,Americas,Americas,EMEA,EMEA,EMEA,Americas,Americas
Territory,None,None,NW America,NW America,NW America,NW America,NW America,NW America,None,None,...,Germany,Germany,Norway,NE America,NE America,Austria,Austria,Austria,NE America,NE America
"Pricing, Delivery_Terms_Quote_Appr",1,0,0,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
"Pricing, Delivery_Terms_Approved",1,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,1,1,1,1
Bureaucratic_Code_0_Approval,1,0,0,1,1,1,1,1,0,0,...,0,0,0,0,0,1,1,1,1,1
Bureaucratic_Code_0_Approved,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,1,1
Bureaucratic_Code,Bureaucratic_Code_4,Bureaucratic_Code_4,Bureaucratic_Code_4,Bureaucratic_Code_5,Bureaucratic_Code_5,Bureaucratic_Code_5,Bureaucratic_Code_5,Bureaucratic_Code_4,Bureaucratic_Code_5,Bureaucratic_Code_5,...,Bureaucratic_Code_4,Bureaucratic_Code_5,Bureaucratic_Code_4,Bureaucratic_Code_5,Bureaucratic_Code_5,Bureaucratic_Code_4,Bureaucratic_Code_4,Bureaucratic_Code_4,Bureaucratic_Code_4,Bureaucratic_Code_4
Account_Created_Date,2015-06-16 00:00:00,2015-06-16 00:00:00,2015-04-21 00:00:00,2013-07-27 00:00:00,2013-07-27 00:00:00,2013-07-27 00:00:00,2013-07-27 00:00:00,2015-04-21 00:00:00,2015-04-20 00:00:00,2015-04-20 00:00:00,...,2013-07-27 00:00:00,2015-12-04 00:00:00,2013-07-27 00:00:00,2014-06-16 00:00:00,2014-06-16 00:00:00,2016-01-15 00:00:00,2016-01-15 00:00:00,2016-01-15 00:00:00,2015-10-20 00:00:00,2015-09-03 00:00:00
Source,None,None,Source_7,Source_11,Source_11,Source_11,Source_11,Source_7,None,None,...,Source_9,None,Source_9,None,None,Source_7,Source_7,Source_7,None,None


In [50]:
pricing_approval = \
    dataset[['Pricing, Delivery_Terms_Quote_Appr', 'Pricing, Delivery_Terms_Approved', 'Stage']]

In [139]:
stages = dataset[['Stage', 'ID']].groupby(['Stage']).count().rename(columns = {'ID': 'count'}).reset_index()
stages

,Stage,count
0,Closed Lost,7350
1,Closed Won,9533
2,Negotiation,18
3,Proposal,35
4,Qualification,11


In [162]:
stages = stages.loc[stages['count'] > 50]
stages['count_vs_total'] = stages['count']/stages['count'].sum()
stages

/home/federico/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Stage,count,count_vs_total
0,Closed Lost,7350,0.435349
1,Closed Won,9533,0.564651


In [163]:
entropia_stages = - (stages['count_vs_total']*np.log2(stages['count_vs_total'])).sum()
entropia_stages

0.9879060159878779

In [164]:
required_approval_stages = dataset[['Stage', 'ID', 'Pricing, Delivery_Terms_Quote_Appr']]\
    .groupby(['Stage', 'Pricing, Delivery_Terms_Quote_Appr']).count().reset_index()\
    .rename(columns = {'ID':'count', 'Closed Won':'Aprobados'})
#required_approval_stages
required_approval_stages = required_approval_stages.pivot_table(index = 'Pricing, Delivery_Terms_Quote_Appr',\
                                    columns = 'Stage',\
                                    values = 'count')\
                                                   .fillna(0).reset_index()
required_approval_stages = required_approval_stages.drop(['Negotiation','Proposal','Qualification'], axis = 1)

required_approval_stages

Stage,"Pricing, Delivery_Terms_Quote_Appr",Closed Lost,Closed Won
0,0,1136.0,2118.0
1,1,6214.0,7415.0


In [ ]:
#entropia de Pricing, Delivery_Terms_Quote_Appr


In [107]:
approval = pd.get_dummies(pricing_approval, \
        columns=['Pricing, Delivery_Terms_Quote_Appr', 'Pricing, Delivery_Terms_Approved'])\
        .groupby('Stage').sum()

In [108]:
approval.T

Stage,Closed Lost,Closed Won,Negotiation,Proposal,Qualification
"Pricing, Delivery_Terms_Quote_Appr_0",1136.0,2118.0,0.0,0.0,0.0
"Pricing, Delivery_Terms_Quote_Appr_1",6214.0,7415.0,18.0,35.0,11.0
"Pricing, Delivery_Terms_Approved_0",3907.0,3129.0,1.0,9.0,11.0
"Pricing, Delivery_Terms_Approved_1",3443.0,6404.0,17.0,26.0,0.0


In [109]:
approval = approval.T

approval['Oportunidades no ganadas'] = \
    approval['Closed Lost'] + approval['Negotiation'] + approval['Proposal'] + approval['Qualification']

approval = approval.rename(columns = {'Closed Won' : 'Oportunidades ganadas'})

approval = approval.drop(columns = ['Closed Lost', 'Negotiation', 'Proposal', 'Qualification'])

approval

Stage,Oportunidades ganadas,Oportunidades no ganadas
"Pricing, Delivery_Terms_Quote_Appr_0",2118.0,1136.0
"Pricing, Delivery_Terms_Quote_Appr_1",7415.0,6278.0
"Pricing, Delivery_Terms_Approved_0",3129.0,3928.0
"Pricing, Delivery_Terms_Approved_1",6404.0,3486.0


In [110]:
oportunidades_totales = approval.sum()
oportunidades_totales

Stage
Oportunidades ganadas       19066.0
Oportunidades no ganadas    14828.0
dtype: float64

In [98]:
oportunidades_totales.sum()

33894.0

In [119]:
resultados_vs_totales = pd.DataFrame(data= oportunidades_totales).T

resultados_vs_totales['Oportunidades ganadas'] = 19066.0/33894.0
resultados_vs_totales['Oportunidades no ganadas'] = 14828.0/33894.0

resultados_vs_totales['entropia'] = -(resultados_vs_totales['Oportunidades ganadas']*np.log2(resultados_vs_totales['Oportunidades ganadas']) + resultados_vs_totales['Oportunidades no ganadas']*np.log2(resultados_vs_totales['Oportunidades no ganadas']))

resultados_vs_totales

Stage,Oportunidades ganadas,Oportunidades no ganadas,entropia
0,0.562518,0.437482,0.988693


In [120]:
approval

Stage,Oportunidades ganadas,Oportunidades no ganadas
"Pricing, Delivery_Terms_Quote_Appr_0",2118.0,1136.0
"Pricing, Delivery_Terms_Quote_Appr_1",7415.0,6278.0
"Pricing, Delivery_Terms_Approved_0",3129.0,3928.0
"Pricing, Delivery_Terms_Approved_1",6404.0,3486.0


In [67]:
approval['ganadas_vs_totales'] = approval['Oportunidades ganadas']/oportunidades_totales.sum()
approval['no ganadas_vs_totales'] = approval['Oportunidades no ganadas']/oportunidades_totales.sum()
approval

Stage,Oportunidades ganadas,Oportunidades no ganadas,ganadas_vs_totales,no ganadas_vs_totales
"Pricing, Delivery_Terms_Quote_Appr_0",2118.0,1136.0,0.062489,0.033516
"Pricing, Delivery_Terms_Quote_Appr_1",7415.0,6278.0,0.218770,0.185225
"Pricing, Delivery_Terms_Approved_0",3129.0,3928.0,0.092317,0.115891
"Pricing, Delivery_Terms_Approved_1",6404.0,3486.0,0.188942,0.102850


In [58]:
approval['entropia'] = \
-(approval['ganadas_vs_totales']*np.log2(approval['ganadas_vs_totales']) + approval['no ganadas_vs_totales']*np.log2(approval['ganadas_vs_totales']))

approval

Stage,Oportunidades ganadas,Oportunidades no ganadas,ganadas_vs_totales,no ganadas_vs_totales,entropia
"Pricing, Delivery_Terms_Quote_Appr_0",2118.0,1136.0,0.062489,0.033516,0.384045
"Pricing, Delivery_Terms_Quote_Appr_1",7415.0,6278.0,0.218770,0.185225,0.885763
"Pricing, Delivery_Terms_Approved_0",3129.0,3928.0,0.092317,0.115891,0.715664
"Pricing, Delivery_Terms_Approved_1",6404.0,3486.0,0.188942,0.102850,0.701464


In [57]:
np.log2(approval['ganadas_vs_totales'])

Pricing, Delivery_Terms_Quote_Appr_0   -4.000255
Pricing, Delivery_Terms_Quote_Appr_1   -2.192511
Pricing, Delivery_Terms_Approved_0     -3.437256
Pricing, Delivery_Terms_Approved_1     -2.403985
Name: ganadas_vs_totales, dtype: float64